In [1]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [2]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Read in helper files

# Enables State Full Name to Abbreviation
state_dict = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/List-of-US-States/master/states.csv', header=None, skiprows=1 , index_col=0, squeeze=True).to_dict()

Files cleaned in order of perceived importance

# Financial Data

### GDP by State and Year

In [4]:
df = pd.read_csv('./dirty/gdp_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [5]:
df.head(2)

,GeoFips,GeoName,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,United States,3,Current-dollar GDP (millions of current dollars),8577552.00,9062817.00,9631172.00,10250952.00,10581929.00,10929108.00,11456450.00,12217196.00,13039197.00,13815583.00,14474228.00,14769862.00,14478067.00,15048970.00,15599731.00,16253970.00,16843196.00,17550687.00,18206023.00,18695106.00,19479623.00,20527159.00,21372582.00,20893746.00
1,0,United States,4,Compensation (millions of dollars),4713220.00,5075701.00,5409937.00,5854634.00,6046346.00,6143370.00,6362298.00,6729306.00,7077722.00,7491260.00,7889371.00,8068682.00,7767191.00,7932970.00,8234017.00,8575362.00,8843637.00,9259654.00,9709535.00,9977096.00,10436745.00,10969807.00,11459449.00,11580088.00


In [6]:
# reshape
df = pd.melt(df, id_vars=['GeoFips','GeoName','LineCode','Description'])

In [7]:
df.sample(5)

,GeoFips,GeoName,LineCode,Description,variable,value
5140,10000,Delaware,3,Current-dollar GDP (millions of current dollars),2014,68354.70
7035,30000,Montana,3,Current-dollar GDP (millions of current dollars),2020,51508.80
5442,10000,Delaware,5,Gross operating surplus (millions of dollars),2015,38102.10
2592,41000,Oregon,5,Gross operating surplus (millions of dollars),2005,55985.00
4227,6000,California,5,Gross operating surplus (millions of dollars),2011,831169.10


In [8]:
# rename
df = df.rename(columns={'variable':'year', 'value':'dollars'}) 

# Map full state names to Abbreviated form
df['state'] = df['GeoName'].map(state_dict)

In [9]:
# drop uneeded columns
df = df.drop(columns=['GeoFips','GeoName','LineCode'])

# Turn GDP Values into 1:1 dollars -- Multiply by 1 million
df['dollars'] = df['dollars']*1000000

# clean up column names a little bit
df['Description'] = df['Description'].str.replace("(millions of dollars)", "", regex=False)
df['Description'] = df['Description'].str.replace("(millions of current dollars)", "", regex=False)

In [10]:
df.sample(5)

,Description,year,dollars,state
69,Subsidies,1997,-142700000.00,ID
4110,Current-dollar GDP,2010,37852500000.00,SD
6268,Taxes on production and imports (TOPI),2017,259440400000.00,NaN
2523,Taxes on production and imports (TOPI),2005,15909600000.00,MN
3472,Gross operating surplus,2008,175612400000.00,NC


In [11]:
df = df.pivot_table(index=['state','year'], # columns to keep
                    columns='Description', # unstack this column
                    values='dollars', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

Description,state,year,Subsidies,Taxes on production and imports (TOPI),Compensation,Current-dollar GDP,Gross operating surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00
2,AK,1999,-56400000.00,1983100000.00,13127500000.00,24744300000.00,9690100000.00
3,AK,2000,-57500000.00,2515400000.00,13893000000.00,26806600000.00,10455700000.00
4,AK,2001,-112300000.00,2385900000.00,14883300000.00,28494100000.00,11337100000.00


In [12]:
# Strip WhiteSpace
df.columns = df.columns.str.strip()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   state                                   1224 non-null   object 
 1   year                                    1224 non-null   object 
 2   Subsidies                               1224 non-null   float64
 3   Taxes on production and imports (TOPI)  1224 non-null   float64
 4   Compensation                            1224 non-null   float64
 5   Current-dollar GDP                      1224 non-null   float64
 6   Gross operating surplus                 1224 non-null   float64
dtypes: float64(5), object(2)
memory usage: 67.1+ KB


In [14]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [15]:
df.head(5)

Description,state,year,subsidies,taxes_on_production_and_imports_(topi),compensation,current-dollar_gdp,gross_operating_surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00
2,AK,1999,-56400000.00,1983100000.00,13127500000.00,24744300000.00,9690100000.00
3,AK,2000,-57500000.00,2515400000.00,13893000000.00,26806600000.00,10455700000.00
4,AK,2001,-112300000.00,2385900000.00,14883300000.00,28494100000.00,11337100000.00


In [16]:
# including `index=False` removes unwanted description level
df.to_csv('./clean/gdp_state_annual_clean.csv', index=False)

### PCE Per Capita -  State Annual

In [17]:
df = pd.read_csv('./dirty/pce_per_capita_state_annual.csv', skipfooter=4, engine='python')
# verified correct head/tail skip against manual excel inspections

In [18]:
df.head(2)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""00000""",United States,,SAPCE2,1,...,Per capita personal consumption expenditures,Dollars,20308,21306,22519,23983,24823,25550,26682,28114,29674,31092,32356,33049,32243,33170,34337,35196,35954,37212,38235,39289,40720,42570,43946,42635
1,"""00000""",United States,,SAPCE2,2,...,Goods,Dollars,7359,7643,8196,8694,8863,9035,9385,9911,10432,10857,11178,11060,10366,10726,11291,11590,11801,12133,12231,12356,12791,13321,13642,14125


In [19]:
# drop initially unnecessary columns
df = df.drop(columns=['Region','TableName','LineCode','IndustryClassification'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoFIPS','GeoName','Description','Unit'])

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

# rename columns
df = df.rename(columns={'variable':'year', 'Description':'pce_description','value':'pce_value'})

# Convert to 1:1 dollars
df['pce_value'] = df['pce_value']

# final drop of unnecessary columns
df = df.drop(columns=['GeoName','Unit','GeoFIPS'])

In [20]:
df.head(5)

,pce_description,year,pce_value,state
0,Per capita personal consumption expenditures,1997,20308,NaN
1,Goods,1997,7359,NaN
2,Durable goods,1997,2624,NaN
3,Motor vehicles and parts,1997,1075,NaN
4,Furnishings and durable household equipment,1997,589,NaN


In [21]:
df = df.pivot_table(index=['year','state'], # columns to keep
                    columns='pce_description', # unstack this column
                    values='pce_value', # populate this this column
                    aggfunc='first'
                    ).reset_index()

In [22]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

In [23]:
df.head(2)

pce_description,year,state,clothing_and_footwear,financial_services_and_insurance,food_and_beverages_purchased_for_off-premises_consumption,food_services_and_accommodations,furnishings_and_durable_household_equipment,gasoline_and_other_energy_goods,gross_output_of_nonprofit_institutions,health_care,housing_and_utilities,less:_receipts_from_sales_of_goods_and_services_by_nonprofit_institutions,motor_vehicles_and_parts,other_durable_goods,other_nondurable_goods,other_services,recreation_services,recreational_goods_and_vehicles,transportation_services,durable_goods,final_consumption_expenditures_of_nonprofit_institutions_serving_households_(npishs),household_consumption_expenditures_(for_services),nondurable_goods,goods,services,per_capita_personal_consumption_expenditures
0,1997,AK,841,1384,2366,1780,614,706,2277,2805,3880,1570,1297,384,1536,1993,1055,909,919,3203,707,13816,5449,8652,14523,23175
1,1997,AL,744,1151,1774,980,496,569,1447,2857,3014,1261,1120,249,1493,1419,650,423,520,2287,186,10591,4580,6867,10777,17645


In [24]:
# add PCE to value columns
cols = df.columns[~df.columns.str.contains('year|state')]

df.rename(columns = dict(zip(cols, 'percap_pce_' + cols)), inplace=True)

In [25]:
df.head(2)

pce_description,year,state,percap_pce_clothing_and_footwear,percap_pce_financial_services_and_insurance,percap_pce_food_and_beverages_purchased_for_off-premises_consumption,percap_pce_food_services_and_accommodations,percap_pce_furnishings_and_durable_household_equipment,percap_pce_gasoline_and_other_energy_goods,percap_pce_gross_output_of_nonprofit_institutions,percap_pce_health_care,percap_pce_housing_and_utilities,percap_pce_less:_receipts_from_sales_of_goods_and_services_by_nonprofit_institutions,percap_pce_motor_vehicles_and_parts,percap_pce_other_durable_goods,percap_pce_other_nondurable_goods,percap_pce_other_services,percap_pce_recreation_services,percap_pce_recreational_goods_and_vehicles,percap_pce_transportation_services,percap_pce_durable_goods,percap_pce_final_consumption_expenditures_of_nonprofit_institutions_serving_households_(npishs),percap_pce_household_consumption_expenditures_(for_services),percap_pce_nondurable_goods,percap_pce_goods,percap_pce_services,percap_pce_per_capita_personal_consumption_expenditures
0,1997,AK,841,1384,2366,1780,614,706,2277,2805,3880,1570,1297,384,1536,1993,1055,909,919,3203,707,13816,5449,8652,14523,23175
1,1997,AL,744,1151,1774,980,496,569,1447,2857,3014,1261,1120,249,1493,1419,650,423,520,2287,186,10591,4580,6867,10777,17645


In [26]:
df.to_csv('./clean/pce_state_annual_clean.csv', index=False)

### Per Capita Personal Income

In [27]:
df = pd.read_csv('./dirty/personal_income_per_capita_state_annual.csv',skiprows=5, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [28]:
df.sample(5)

,GeoFips,GeoName,LineCode,Description,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
108,39000,Ohio,1,Personal income (millions of dollars),5113.9,4421.8,3772,2704.2,2608.5,3083.1,3520,4060.5,4434.3,3864.9,4268.3,4600.5,5759,7190.4,8740.5,9212.6,9414.6,9952.6,10873.5,12137.5,11610.5,12855.10,14837.90,15911.70,17450.00,17513.20,18913.70,20191.20,21231.90,21063.00,22540.20,23455.40,23909.90,25206.80,26265.40,28027.90,30301.80,33056.90,34629.90,38206.80,41801.00,44141.90,46980.40,50967.90,56552.40,61925.20,66162.20,72597.50,80397.20,88854.70,98136.00,107184.50,116652.40,122081.50,129132.50,142661.10,151425.60,158493.00,166090.30,178267.50,191708.30,202979.60,207676.90,221201.40,230366.40,243306.80,255101.10,267147.20,282809.00,299095.80,308329.30,324977.50,333912.20,338738.30,348858.50,362572.60,373789.40,392816.90,409745.80,424081.90,412219.00,425361.80,453595.40,471703.90,479088.70,498894.60,518826.80,529069.60,548272.70,569766.10,587177.30,627231.30,665374.40
18,8000,Colorado,1,Personal income (millions of dollars),635.6,596.8,498.5,380.3,378.4,399.5,480,590.4,589.5,564.5,581.4,619.7,736,1038.5,1228.2,1221.3,1350.2,1472.3,1695.8,1880.6,1914.6,2098.90,2493.80,2692.50,2725.80,2808.70,3059.70,3376.00,3699.10,3787.10,4068.00,4353.40,4683.90,4942.10,5174.40,5474.20,5840.60,6275.80,6763.50,7502.70,8438.80,9540.20,10759.70,12176.10,13923.10,15609.20,17249.70,18981.30,21174.20,24413.50,27979.30,32050.80,36882.90,40505.90,43694.20,47644.60,50483.60,52445.70,54400.00,57556.60,62010.10,65667.20,69601.90,75599.10,81812.10,87983.80,95648.80,103235.80,111207.80,123090.00,132293.90,147242.40,154444.50,154639.60,157851.90,163872.90,175069.20,189465.50,202115.90,210029.40,199351.60,205866.10,223492.60,236759.10,249512.70,271410.20,284836.80,289673.00,309657.70,331955.40,350390.10,370392.10,401122.90
162,93000,Great Lakes,1,Personal income (millions of dollars),20057.6,17176.9,14355.4,10479.1,9677.7,11559,13400,15431,17159.7,15071.3,16478.2,17843.6,22162.6,27414.4,33230.5,35182.4,35933.9,38799.9,42792.6,47750.9,45996.4,51213.10,58155.30,61664.80,67239.70,66584.90,72014.00,77180.90,80391.30,80303.90,85593.80,88933.50,91138.50,96685.20,101230.50,108769.50,118705.60,129095.60,135847.30,148258.00,162016.50,170705.30,183694.70,200707.90,224429.10,243765.60,263253.10,289628.30,321433.80,356577.60,392787.60,425780.60,464803.90,486801.30,512202.90,565477.90,600323.90,632500.10,664885.90,712815.80,767369.60,810354.10,830740.60,890729.10,930987.70,987885.50,1041548.90,1099478.70,1161498.60,1234434.60,1285398.20,1370277.40,1411718.20,1427690.60,1464134.40,1522011.00,1573033.80,1654969.80,1728248.10,1783643.60,1716691.10,1776452.20,1879160.00,1960700.30,1991993.70,2079147.10,2171984.30,2213769.80,2286479.00,2389121.50,2465917.50,2625186.90,2801365.20
173,96000,Southwest,3,Per capita personal income (dollars) 2/,470,398,333,250,246,282,320,362,409,390,401,423,517,726,931,1039,1061,1038,1125,1211,1287,1341.00,1489.00,1582.00,1622.00,1648.00,1717.00,1804.00,1880.00,1927.00,2004.00,2024.00,2088.00,2143.00,2208.00,2339.00,2488.00,2679.00,2885.00,3158.00,3460.00,3763.00,4006.00,4350.00,4825.00,5309.00,5858.00,6403.00,6988.00,7864.00,8816.00,9811.00,11169.00,11844.00,12292.00,13260.00,14051.00,14240.00,14512.00,15325.00,16232.00,17157.00,17601.00,18541.00,19196.00,19896.00,20806.00,21876.00,23217.00,24685.00,25525.00,27285.00,28479.00,28542.00,29360.00,30611.00,32715.00,35107.00,36448.00,38807.00,36249.00,37682.00,40289.00,42278.00,42919.00,45311.00,45532.00,44935.00,47278.00,49815.00,51449.00,53755.00,57647.00
9,4000,Arizona,1,Personal income (millions

In [29]:
# drop un-needed columns
df = df.drop(columns=['LineCode','GeoFips'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoName','Description'])

# rename columns
df = df.rename(columns={'variable':'year'})

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

In [30]:
df.sample(2)

,GeoName,Description,year,value,state
394,Georgia,Population (persons) 1/,1931,2924000,GA
1968,Southeast,Personal income (millions of dollars),1939,9582.4,NaN


In [31]:
df.Description.value_counts()

Personal income (millions of dollars)      5580
Population (persons) 1/                    5580
Per capita personal income (dollars) 2/    5580
Name: Description, dtype: int64

In [32]:
# filter to only Per Capita Personal Income
df = df.loc[(df['Description']=='Per capita personal income (dollars) 2/') | (df['Description']=='Population (persons) 1/')]

In [33]:
# Clean up categorical values
df['Description'] = df['Description'].str.replace(' (persons) 1/','', regex=False)
df['Description'] = df['Description'].str.replace(' (dollars) 2/','', regex=False)

In [34]:
# remove nulls generated by region drop from state mapping
df = df.dropna()

df = df.drop(columns='GeoName')

In [35]:
df.head(5)

,Description,year,value,state
4,Population,1929,2644000,AL
5,Per capita personal income,1929,319,AL
10,Population,1929,430000,AZ
11,Per capita personal income,1929,598,AZ
13,Population,1929,1852000,AR


In [36]:
df = df.pivot_table(index=['year','state'], # columns to keep
                    columns='Description', # unstack this column
                    values='value', # populate this this column
                    aggfunc='first'
                    ).reset_index()

In [39]:
df.to_csv('./clean/per_capita_personal_income-population_state_annual_clean.csv', index=False)

### Business Applications State Annual

In [40]:
df = pd.read_csv('./dirty/business_apps_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

# Data exists from 2005-2021

In [41]:
# NAIC Sector Codes
naic_sector_codes = pd.read_csv('./supplemental_data/naic_sectors_codes.csv', skiprows=0 , index_col=0, squeeze=True).to_dict()

# NAIC Series Codes
naic_series_codes = pd.read_csv('./supplemental_data/naic_series_codes.csv', skiprows=0 , index_col=0, squeeze=True).to_dict()

In [42]:
# Filter out incomplete years
df = df[df['year']!=2022]

df = df[df['year']!=2004]

# filter out unwanted series
df = df[~df['series'].isin(['BF_PBF4Q','BF_PBF8Q','BF_SBF4Q','BF_SBF8Q'])]

# filter to only seasonally adjusted data
df = df[df['sa']=='A']

In [43]:
df.tail(5)

,sa,naics_sector,series,geo,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
27275,A,TOTAL,BF_BF8Q,VA,2005,1070,1090,1043,1054,1036,1088,1009,1021,1044,1130,1103,1084
27277,A,TOTAL,BF_BF8Q,WA,2005,1161,1063,1078,1060,993,1109,1162,1156,840,1230,1197,1159
27279,A,TOTAL,BF_BF8Q,WV,2005,D,161,D,148,160,187,167,D,D,201,202,172
27281,A,TOTAL,BF_BF8Q,WI,2005,525,620,609,645,606,624,622,598,467,620,593,553
27283,A,TOTAL,BF_BF8Q,WY,2005,D,D,121,120,119,125,D,D,D,D,D,D


In [44]:
# convert selected columns by index(months) to numeric
df.iloc[:,5:17] = df.iloc[:,5:17].apply(pd.to_numeric, errors='coerce')

# create annual total
df['annual_total'] =  df.iloc[:,5:17].sum(axis=1, min_count=12)

# drop monthly columns after creating annual total
df = df.drop(df.columns[5:17], axis=1)

In [45]:
df.sample(5)

,sa,naics_sector,series,geo,year,annual_total
16587,A,TOTAL,BA_CBA,MI,2011,13043.00
4895,A,NAICS52,BA_BA,US,2018,214653.00
3678,A,NAICS51,BA_BA,US,2019,72999.00
13221,A,TOTAL,BA_CBA,HI,2013,1536.00
16771,A,TOTAL,BA_HBA,PA,2011,30875.00


In [46]:
# Map supplemental NAIC Data to columns
df['naics_sector'] = df['naics_sector'].map(naic_sector_codes)

df['series'] = df['series'].map(naic_series_codes)

In [47]:
df = df.pivot_table(index=['sa','geo','year','naics_sector'], # columns to keep
                    columns='series', # unstack this column
                    values='annual_total', # populate this this column
                    aggfunc='first'
                    ).reset_index()

df.head(5)

series,sa,geo,year,naics_sector,Business Applications,Business Applications from Corporations,Business Formations within Eight Quarters,Business Formations within Four Quarters,High‐Propensity Business Applications BA_WBA – Business Applications with Planned Wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,NaN,1152.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,NaN,1028.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,NaN,953.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,NaN,824.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,NaN,742.00,2122.00


In [48]:
df.head(5)

series,sa,geo,year,naics_sector,Business Applications,Business Applications from Corporations,Business Formations within Eight Quarters,Business Formations within Four Quarters,High‐Propensity Business Applications BA_WBA – Business Applications with Planned Wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,NaN,1152.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,NaN,1028.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,NaN,953.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,NaN,824.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,NaN,742.00,2122.00


In [49]:
df.to_csv('./clean/business_apps_state_annual_clean.csv', index=False)

### Poverty Rate

In [50]:
df = pd.read_csv('./dirty/poverty_rate_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [51]:
df.sample(2)

,STATE,Percent,Year,Unnamed: 3
1163,South Dakota,10.80,1998,NaN
776,Hawaii,8.60,2005,NaN


In [52]:
# drop unused columns
df = df.drop(columns='Unnamed: 3')

# convert to numeric percent
df['Percent'] = (df['Percent']/100)

# Map states to abbreviated form
df['STATE'] = df['STATE'].map(state_dict)

# convert column names to lower case
df.columns = df.columns.str.lower()

df = df.rename(columns={'percent':'poverty_rate'})

In [53]:
df.sample(2)

,state,poverty_rate,year
331,MO,0.10,2014
831,IA,0.11,2004


In [54]:
df.to_csv('./clean/poverty_rate_state_annual_clean.csv', index=False)

# Education Data

### Total Spending per Student

In [55]:
df = pd.read_csv('./dirty/tot_spend_student_state_annual.csv', skiprows=6, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [56]:
# Create String
string = 'Total Current Expenditures for Public El-Sec (TE5) per Pupil (MEMBR) [State Finance]'

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

In [57]:
df.sample(1)

,State Name,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,2012-13,2011-12,2010-11,2009-10,2008-09,2007-08,2006-07,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98
4,CALIFORNIA,13831,12664,12151,11420,10449,9671,9258,9329,9146,9300,9503,9706,8952,8301,7905,7673,7552,7405,6987,6314,5801,5644


In [58]:
df['State Name'].unique().shape

(51,)

In [59]:
# reshape dataframe
df = pd.melt(df, id_vars=['State Name'])

#rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_expenditure'})

# trim to desired year digits
df['start_year'] = df['start_year'].str[:5]

# convert state dictionary to uppercase values - match dataframe format
state_dict_upper = {k.upper():v.upper() for k,v in state_dict.items()}

# map state name to state abbreviation
df['state'] = df['state'].map(state_dict_upper)

In [60]:
# Check output
df.head(5)

,state,start_year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831


In [61]:
df.state.unique().shape

(51,)

In [62]:
df.to_csv('./clean/tot_spend_student_state_annual_clean.csv', index=False)

### Total Revenue Per Student

In [63]:
df = pd.read_csv('./dirty/tot_rev_per_student_state_annual.csv', skiprows=0, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [64]:
df.head(5)

,State Name,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2018-19,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2017-18,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2016-17,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2015-16,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2014-15,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2013-14,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2012-13,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2011-12,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2010-11,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2009-10,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2008-09,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2007-08,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2006-07,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2005-06,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2004-05,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2003-04,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2002-03,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2001-02,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2000-01,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1999-00,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1998-99,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1997-98
0,ALABAMA,11415,10792,10590,10200,9992,9913,9653,9534,9776,9667,9708,10356,9548,8555,8028,7349,6971,6956,6503,6523,5975,5535
1,ALASKA,19553,19038,18897,18851,22379,20447,20312,19034,18699,17759,17319,17471,14304,12849,12632,11576,10928,10717,10275,10118,9532,9222
2,ARIZONA,10396,9697,9214,9293,8995,8703,8616,8613,9111,9343,9002,9457,9023,8071,7814,7550,7791,7214,7071,6455,5988,5812
3,ARKANSAS,11733,11592,11388,10939,10762,10478,10391,10939,10939,10738,10072,9758,9362,9031,8712,7542,7243,7112,6250,6054,5772,5697
4,CALIFORNIA,16014,14535,13965,13606,11786,10985,10481,10590,10790,10239,11180,11228,10857,9909,9234,8980,8975,8363,8306,7465,6750,6572


In [65]:
# Create String
string = 'Total Revenues (TR) per Pupil (MEMBR) [State Finance] '

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

# Reshape df
df = pd.melt(df, id_vars=['State Name'])

# Rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_revenue'})

# remap state name to state abbreviations
df['state'] = df['state'].map(state_dict_upper)

# Use Start year instead of year-span
df['start_year'] = df['start_year'].str[:4]

In [66]:
# Sample output
df.sample(10)

,state,start_year,per_pupil_revenue
402,VT,2011,18288
845,NH,2002,9425
749,OH,2004,10822
69,LA,2017,12772
447,RI,2010,15846
118,KS,2016,12833
851,OH,2002,9870
219,IA,2014,12791
824,DC,2002,14626
1046,MT,1998,6546


In [67]:
df.to_csv('./clean/tot_revenue_student_state_annual_clean.csv', index=False)